![Team USA Football](images/Usa_football_body_logo.png)

### Overview

This projects mission was to build a model that can determine what contributes to a football players sucess on the field.

### Business Understanding

Foxy Stats was hired to perform data analysis for Team USA. Flag Football is coming to the 2028 Olympic Games in LA and they need the best our country has to offer. While the NFL is full contact
they are still some of the players in the United States. We are to find the best of the best to help fill in some gaps in the team 

### Data Understanding and Limitations
The data comes from nflverse via their [GitHub](https://github.com/nflverse/nflverse-data/releases). The initial data contains offensive stats from 1999 to 2023. It is updated on a weekly basis during the football season and after any stat is corrected as well.  

Data Dictionary available to help better understand these stats [here](https://nflreadr.nflverse.com/articles/dictionary_player_stats.html)

Limitations include no record of win or loss. No record of defensive stats to paint a fuller picture. No record of lineman performance upon a sack or rushing negative yards, fumble, etc  

### Table of Contents
- Exploratory Data Analysis
    - Feature Engineering
- Modeling and Evaluating
    - DecisionTree
    - Random Forest Classifier (RFC)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

ill save my data as some new dataframes and see what i have.

In [ ]:
stats_df = pd.read_csv('data/player_stats.csv')

ill start with stats_df as that will require more work

In [ ]:
stats_df.head()

In [ ]:
# lets check up on Aaron Rodgers
stats_df[stats_df['player_display_name'] == 'Aaron Rodgers'].head()

In [ ]:
stats_df.info()

In [ ]:
# Use 1 to show all info in cell below then comment it out and use 2 to go back to default view I can delete this later

#1
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

#2
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_colwidth')


In [ ]:
stats_df['position'].value_counts()

ive only got offensive related stats going back to 1999. I wont be needing those older players in a team for today

In [ ]:
# look at the last ten years of data 
stats_df = stats_df[stats_df['season'].isin(range(2014, 2024))]

In [ ]:
# look at the regular season so we can see every player in action. this is about players not teams
stats_df = stats_df[stats_df['season_type'] == 'REG']


In [ ]:
#working solely with the offense seperate by position for our 3 analyses
# Filter for quarterbacks
qbs_df = stats_df[stats_df['position'] == 'QB']

# Filter for rushers (RBs and FBs)
rushers_df = stats_df[stats_df['position'].isin(['RB', 'FB', 'HB'])]

# Filter for receivers (WRs and TEs)
receivers_df = stats_df[stats_df['position'].isin(['WR', 'TE'])]

In [ ]:
# QB metrics
qbs_df.loc[:, 'completion_percentage'] = qbs_df['completions'] / qbs_df['attempts']
qbs_df.loc[:, 'yards_per_attempt'] = qbs_df['passing_yards'] / qbs_df['attempts']
qbs_df.loc[:, 'pass_tds_per_attempt'] = qbs_df['passing_tds'] / qbs_df['attempts']
qbs_df.loc[:, 'season_passing_yards'] = qbs_df.groupby(['player_id', 'season'])['passing_yards'].transform('sum')
qbs_df.loc[:, 'season_passing_tds'] = qbs_df.groupby(['player_id', 'season'])['passing_tds'].transform('sum')
qbs_df.loc[:, 'season_pass_attempts'] = qbs_df.groupby(['player_id', 'season'])['attempts'].transform('sum')
qbs_df.loc[:, 'season_completions'] = qbs_df.groupby(['player_id', 'season'])['completions'].transform('sum')
qbs_df.loc[:, 'season_completion_percentage'] = qbs_df.groupby(['player_id', 'season'])['completion_percentage'].transform('mean')
qbs_df.loc[:, 'season_yards_per_attempt'] = qbs_df.groupby(['player_id', 'season'])['yards_per_attempt'].transform('mean')
qbs_df.loc[:, 'season_pass_tds_per_attempt'] = qbs_df.groupby(['player_id', 'season'])['pass_tds_per_attempt'].transform('mean')

# Rusher metrics
rushers_df.loc[:, 'yards_per_carry'] = rushers_df['rushing_yards'] / rushers_df['carries']
rushers_df.loc[:, 'tds_per_carry'] = rushers_df['rushing_tds'] / rushers_df['carries']
rushers_df.loc[:, 'season_rushing_tds'] = rushers_df.groupby(['player_id', 'season'])['rushing_tds'].transform('sum')
rushers_df.loc[:, 'season_rushing_yards'] = rushers_df.groupby(['player_id', 'season'])['rushing_yards'].transform('sum')
rushers_df.loc[:, 'season_carries'] = rushers_df.groupby(['player_id', 'season'])['carries'].transform('sum')
rushers_df.loc[:, 'season_yards_per_carry'] = rushers_df.groupby(['player_id', 'season'])['yards_per_carry'].transform('mean')
rushers_df.loc[:, 'season_tds_per_carry'] = rushers_df.groupby(['player_id', 'season'])['tds_per_carry'].transform('mean')

# Receiver metrics
receivers_df.loc[:, 'yards_per_reception'] = receivers_df['receiving_yards'] / receivers_df['receptions']
receivers_df.loc[:, 'season_receiving_yards'] = receivers_df.groupby(['player_id', 'season'])['receiving_yards'].transform('sum')
receivers_df.loc[:, 'tds_per_reception'] = receivers_df['receiving_tds'] / receivers_df['receptions']
receivers_df.loc[:, 'season_receiving_tds'] = receivers_df.groupby(['player_id', 'season'])['receiving_tds'].transform('sum')
receivers_df.loc[:, 'season_receptions'] = receivers_df.groupby(['player_id', 'season'])['receptions'].transform('sum')
receivers_df.loc[:, 'season_yards_per_reception'] = receivers_df.groupby(['player_id', 'season'])['yards_per_reception'].transform('mean')
receivers_df.loc[:, 'season_tds_per_reception'] = receivers_df.groupby(['player_id', 'season'])['tds_per_reception'].transform('mean')

## Qbs

In [ ]:
qbs_df = qbs_df.drop(columns= ['player_id',
                               'player_name',
                               'position', 
                               'position_group',
                               'headshot_url',
                               'season_type',   
                               'passing_epa',
                               'pacr',
                               'dakota',
                               'attempts',
                               'completions',
                               'passing_yards',
                               'passing_tds',
                               'interceptions',
                               'sacks',
                               'sack_yards',
                               'sack_fumbles',
                               'sack_fumbles_lost',
                               'passing_air_yards',
                               'passing_yards_after_catch',
                               'passing_first_downs',
                               'passing_epa',
                               'passing_2pt_conversions',
                               'carries',
                               'rushing_yards',
                               'rushing_tds',
                               'rushing_fumbles',
                               'rushing_fumbles_lost',
                               'rushing_first_downs',
                               'rushing_epa',
                               'rushing_2pt_conversions',
                               'receptions',
                               'targets',
                               'receiving_yards',
                               'receiving_tds',
                               'receiving_fumbles',
                               'receiving_fumbles_lost',
                               'receiving_air_yards',
                               'receiving_yards_after_catch',
                               'receiving_first_downs',
                               'receiving_epa',
                               'receiving_2pt_conversions',
                               'racr',
                               'target_share',
                               'air_yards_share',
                               'wopr',
                               'special_teams_tds',
                               'fantasy_points',
                               'fantasy_points_ppr'
                              ]).reset_index(drop=True)
qbs_df.head()

In [ ]:
# Get the index of the maximum week for each player
max_week_idx = qbs_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
qbs_df = qbs_df.loc[max_week_idx].reset_index(drop=True)
qbs_df = qbs_df.drop(columns= 'week')
qbs_df.head()

In [ ]:
qbs_df = qbs_df.dropna(subset=['completion_percentage'])

In [ ]:
qbs_df = qbs_df[qbs_df['season_pass_attempts'] >= 100]
qbs_df.drop(columns=['recent_team', 'opponent_team', 'completion_percentage', 'yards_per_attempt', 'pass_tds_per_attempt'], inplace=True)

In [ ]:
qbs_df.head()

In [ ]:
qbs_df.info()

## Rushers

In [ ]:
rushers_df = rushers_df.drop(columns= ['player_id',
                                       'player_name',
                                       'position', 
                                       'position_group',
                                       'headshot_url',
                                       'season_type',
                                       'passing_epa',
                                       'pacr',
                                       'dakota',
                                       'completions',
                                       'attempts',
                                       'passing_yards',
                                       'passing_tds',
                                       'interceptions',
                                       'sacks',
                                       'sack_yards',
                                       'sack_fumbles',
                                       'sack_fumbles_lost',
                                       'passing_air_yards',
                                       'passing_yards_after_catch',
                                       'passing_first_downs',
                                       'passing_epa',
                                       'passing_2pt_conversions',
                                       'rushing_epa',
                                       'receptions',
                                       'targets',
                                       'receiving_yards',
                                       'receiving_tds',
                                       'receiving_fumbles',
                                       'receiving_fumbles_lost',
                                       'receiving_air_yards',
                                       'receiving_yards_after_catch',
                                       'receiving_first_downs',
                                       'receiving_epa',
                                       'receiving_2pt_conversions',
                                       'racr',
                                       'target_share',
                                       'air_yards_share',
                                       'wopr',
                                       'special_teams_tds',
                                       'fantasy_points',
                                       'fantasy_points_ppr'
                                      ]).reset_index(drop=True)

rushers_df.head()

In [ ]:
rushers_df.info()

In [ ]:
# Get the index of the maximum week for each player
max_week_idx = rushers_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
rushers_df = rushers_df.loc[max_week_idx].reset_index(drop=True)
rushers_df = rushers_df.drop(columns= 'week')
rushers_df.head()

In [ ]:
rushers_df = rushers_df[rushers_df['season_carries'] >= 100]
rushers_df.drop(columns=['recent_team', 'opponent_team', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_2pt_conversions', 'yards_per_carry', 'tds_per_carry'], inplace=True)

In [ ]:
rushers_df.head()

In [ ]:
rushers_df.info()

## Receivers

In [ ]:
receivers_df = receivers_df.drop(columns= ['player_id',
                                           'player_name',
                                           'position', 
                                           'position_group',
                                           'headshot_url',
                                           'season_type',
                                           'passing_epa',
                                           'pacr',
                                           'dakota',
                                           'completions',
                                           'attempts',
                                           'passing_yards',
                                           'passing_tds',
                                           'interceptions',
                                           'sacks',
                                           'sack_yards',
                                           'sack_fumbles',
                                           'sack_fumbles_lost',
                                           'passing_air_yards',
                                           'passing_yards_after_catch',
                                           'passing_first_downs',
                                           'passing_epa',
                                           'passing_2pt_conversions',
                                           'carries',
                                           'rushing_yards',
                                           'rushing_tds',
                                           'rushing_fumbles',
                                           'rushing_fumbles_lost',
                                           'rushing_first_downs',
                                           'rushing_epa',
                                           'rushing_2pt_conversions',
                                           'receiving_epa',
                                           'racr',
                                           'target_share',
                                           'air_yards_share',
                                           'wopr',
                                           'special_teams_tds',
                                           'fantasy_points',
                                           'fantasy_points_ppr'
                                          ]).reset_index(drop=True)

receivers_df.head()

In [ ]:
receivers_df.info()

In [ ]:
# Get the index of the maximum week for each player
max_week_idx = receivers_df.groupby(['player_display_name', 'season'])['week'].idxmax()

# Select the rows corresponding to these indices
receivers_df = receivers_df.loc[max_week_idx].reset_index(drop=True)
receivers_df = receivers_df.drop(columns= 'week')
receivers_df.head()

In [ ]:
receivers_df = receivers_df[receivers_df['season_receptions'] >= 75]
receivers_df.drop(columns=['recent_team', 'opponent_team', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_2pt_conversions', 'yards_per_reception', 'tds_per_reception'], inplace=True)

In [ ]:
receivers_df.head()

In [ ]:
receivers_df.info()

# Modeling

## Starting with Qbs

In [ ]:
# Features (excluding the target and the player name)
X = qbs_df[['season', 'season_passing_yards', 'season_pass_attempts', 
            'season_completion_percentage', 'season_yards_per_attempt', 
            'season_pass_tds_per_attempt']]
# Target variable
y = qbs_df['season_passing_tds']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Dummy

In [ ]:
# Initialize the dummy model
dummy_model = DummyRegressor(strategy='mean')

# Train the model
dummy_model.fit(X_train, y_train)

# Make predictions
y_dummy_pred = dummy_model.predict(X_test)

# Calculate mean squared error
dummy_mse = mean_squared_error(y_test, y_dummy_pred)
print(f'Dummy Model Mean Squared Error: {dummy_mse}')

### Decision Tree

In [ ]:
# Initialize the decision tree model
tree_model = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model.fit(X_train, y_train)

# Make predictions
y_tree_pred = tree_model.predict(X_test)

# Calculate mean squared error
tree_mse = mean_squared_error(y_test, y_tree_pred)
print(f'Decision Tree Model Mean Squared Error: {tree_mse}')

In [ ]:
# Get feature importances
feature_importances = tree_model.feature_importances_

# Create a DataFrame for better visualization
feature_importances_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(feature_importances_df)


### Random Forest

In [ ]:
# Cross-validation with RandomForest
rf_model = RandomForestRegressor(random_state=42)
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated Mean Squared Error: {-cv_scores.mean()}')

In [ ]:
# Train the model on the entire dataset
rf_model.fit(X_train, y_train)
importances = rf_model.feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
print(feature_importances_df)

### Grid Search + Random Forest

In [ ]:
# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Set up the grid search
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

In [ ]:
# Perform the grid search
grid_search.fit(X_train, y_train)

In [ ]:
# Get the best parameters
best_params = grid_search.best_params_
print(f'Best parameters found: {best_params}')

# Get the best estimator
best_rf_model = grid_search.best_estimator_

# Train the best model on the entire dataset
best_rf_model.fit(X_train, y_train)
importances_best = best_rf_model.feature_importances_
feature_importances_best_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances_best
}).sort_values(by='Importance', ascending=False)
print(feature_importances_best_df)

In [ ]:
# Make predictions with the best model
y_best_pred = best_rf_model.predict(X_test)

# Calculate metrics
best_mse = mean_squared_error(y_test, y_best_pred)
best_rmse = mean_squared_error(y_test, y_best_pred, squared=False)
best_mae = mean_absolute_error(y_test, y_best_pred)
best_r2 = r2_score(y_test, y_best_pred)

print(f'Best Random Forest Model Mean Squared Error (MSE): {best_mse}')
print(f'Best Random Forest Model Root Mean Squared Error (RMSE): {best_rmse}')
print(f'Best Random Forest Model Mean Absolute Error (MAE): {best_mae}')
print(f'Best Random Forest Model R-squared (R²): {best_r2}')

the overwhelmingly most important thing to a QB getting the most touchdowns in a season is getting the most passing yards in a season therefore our reccomendations will be the top QB in passing yards and TDs

## Rushers

In [ ]:
rushers_df.head()

In [ ]:
# Features (excluding the target and the player name)
X1 = rushers_df[['season', 'season_rushing_yards', 'season_carries', 
            'season_yards_per_carry', 'season_tds_per_carry']]
# Target variable
y1 = rushers_df['season_rushing_tds']

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

### Dummy

In [ ]:
# Initialize the dummy model
dummy1_model = DummyRegressor(strategy='mean')

# Train the model
dummy1_model.fit(X1_train, y1_train)

# Make predictions
y1_dummy_pred = dummy1_model.predict(X1_test)

# Calculate mean squared error
dummy1_mse = mean_squared_error(y1_test, y1_dummy_pred)
print(f'Dummy Model Mean Squared Error: {dummy1_mse}')

### Decision Tree

In [ ]:
# Initialize the decision tree model
tree_model1 = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model1.fit(X1_train, y1_train)

# Make predictions
y1_tree_pred = tree_model1.predict(X1_test)

# Calculate mean squared error
tree_mse1 = mean_squared_error(y1_test, y1_tree_pred)
print(f'Decision Tree Model Mean Squared Error: {tree_mse1}')

In [ ]:
# Get feature importances
feature_importances1 = tree_model1.feature_importances_

# Create a DataFrame for better visualization
feature_importances1_df = pd.DataFrame({
    'Feature': X1.columns,
    'Importance': feature_importances1
}).sort_values(by='Importance', ascending=False)

print(feature_importances1_df)


### Random Forest

In [ ]:
# Cross-validation with RandomForest
rf_model1 = RandomForestRegressor(random_state=42)
cv_scores1 = cross_val_score(rf_model1, X1_train, y1_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated Mean Squared Error: {-cv_scores1.mean()}')

In [ ]:
# Train the model on the entire dataset
rf_model1.fit(X1_train, y1_train)
importances1 = rf_model1.feature_importances_
feature_importances1_df = pd.DataFrame({
    'Feature': X1.columns,
    'Importance': importances1
}).sort_values(by='Importance', ascending=False)
print(feature_importances1_df)

### Grid Search + Random Forest

In [ ]:
# Set up the grid search
grid_search1 = GridSearchCV(estimator=rf_model1, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

In [ ]:
# Perform the grid search
grid_search1.fit(X1_train, y1_train)

In [ ]:
# Get the best parameters
best_params1 = grid_search1.best_params_
print(f'Best parameters found: {best_params1}')

# Get the best estimator
best_rf_model1 = grid_search1.best_estimator_

# Train the best model on the entire dataset
best_rf_model1.fit(X1_train, y1_train)
importances_best1 = best_rf_model1.feature_importances_
feature_importances_best_df1 = pd.DataFrame({
    'Feature': X1.columns,
    'Importance': importances_best1
}).sort_values(by='Importance', ascending=False)
print(feature_importances_best_df1)

In [ ]:
# Make predictions with the best model
y1_best_pred = best_rf_model1.predict(X1_test)

# Calculate metrics
best_mse1 = mean_squared_error(y1_test, y1_best_pred)
best_rmse1 = mean_squared_error(y1_test, y1_best_pred, squared=False)
best_mae1 = mean_absolute_error(y1_test, y1_best_pred)
best_r21 = r2_score(y1_test, y1_best_pred)

print(f'Best Random Forest Model Mean Squared Error (MSE): {best_mse1}')
print(f'Best Random Forest Model Root Mean Squared Error (RMSE): {best_rmse1}')
print(f'Best Random Forest Model Mean Absolute Error (MAE): {best_mae1}')
print(f'Best Random Forest Model R-squared (R²): {best_r21}')

tds per carry and total yards no suprise there

## Receivers

In [ ]:
receivers_df.head()

In [ ]:
# Features (excluding the target and the player name)
X2 = receivers_df[['season', 'season_receiving_yards', 'season_receptions', 
            'season_yards_per_reception', 'season_tds_per_reception']]
# Target variable
y2 = receivers_df['season_receiving_tds']

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

### Dummy

In [ ]:
# Initialize the dummy model
dummy2_model = DummyRegressor(strategy='mean')

# Train the model
dummy2_model.fit(X2_train, y2_train)

# Make predictions
y2_dummy_pred = dummy2_model.predict(X2_test)

# Calculate mean squared error
dummy2_mse = mean_squared_error(y2_test, y2_dummy_pred)
print(f'Dummy Model Mean Squared Error: {dummy2_mse}')

### Decision Tree

In [ ]:
# Initialize the decision tree model
tree_model2 = DecisionTreeRegressor(random_state=42)

# Train the model
tree_model2.fit(X2_train, y2_train)

# Make predictions
y2_tree_pred = tree_model2.predict(X2_test)

# Calculate mean squared error
tree_mse2 = mean_squared_error(y2_test, y2_tree_pred)
print(f'Decision Tree Model Mean Squared Error: {tree_mse2}')

In [ ]:
# Get feature importances
feature_importances2 = tree_model2.feature_importances_

# Create a DataFrame for better visualization
feature_importances2_df = pd.DataFrame({
    'Feature': X2.columns,
    'Importance': feature_importances2
}).sort_values(by='Importance', ascending=False)

print(feature_importances2_df)

### Random Forest

In [ ]:
# Cross-validation with RandomForest
rf_model2 = RandomForestRegressor(random_state=42)
cv_scores2 = cross_val_score(rf_model2, X2_train, y2_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-validated Mean Squared Error: {-cv_scores2.mean()}')

In [ ]:
# Train the model on the entire dataset
rf_model2.fit(X2_train, y2_train)
importances2 = rf_model2.feature_importances_
feature_importances2_df = pd.DataFrame({
    'Feature': X2.columns,
    'Importance': importances2
}).sort_values(by='Importance', ascending=False)
print(feature_importances2_df)

### Grid Search + Random Forest

In [ ]:
# Set up the grid search
grid_search2 = GridSearchCV(estimator=rf_model2, param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)

In [ ]:
# Perform the grid search
grid_search2.fit(X2_train, y2_train)

In [ ]:
# Get the best parameters
best_params2 = grid_search2.best_params_
print(f'Best parameters found: {best_params2}')

# Get the best estimator
best_rf_model2 = grid_search2.best_estimator_

# Train the best model on the entire dataset
best_rf_model2.fit(X2_train, y2_train)
importances_best2 = best_rf_model2.feature_importances_
feature_importances_best_df2 = pd.DataFrame({
    'Feature': X2.columns,
    'Importance': importances_best2
}).sort_values(by='Importance', ascending=False)
print(feature_importances_best_df2)

In [ ]:
# Make predictions with the best model
y2_best_pred = best_rf_model2.predict(X2_test)

# Calculate metrics
best_mse2 = mean_squared_error(y2_test, y2_best_pred)
best_rmse2 = mean_squared_error(y2_test, y2_best_pred, squared=False)
best_mae2 = mean_absolute_error(y2_test, y2_best_pred)
best_r22 = r2_score(y2_test, y2_best_pred)

print(f'Best Random Forest Model Mean Squared Error (MSE): {best_mse2}')
print(f'Best Random Forest Model Root Mean Squared Error (RMSE): {best_rmse2}')
print(f'Best Random Forest Model Mean Absolute Error (MAE): {best_mae2}')
print(f'Best Random Forest Model R-squared (R²): {best_r22}')

tds per reception and yards again. similar to rbs 